In [1]:
import os
os.getcwd()

'/home/dhcoe/projects/mlcourse/src'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import os
from sklearn.metrics import log_loss

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [12]:
def evaluate_mdl(X,y):
    
    scorer=make_scorer(log_loss, greater_is_better=False)

    params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
    grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=0, cv=3,n_jobs=-1 ,scoring=scorer)
    grid_search_cv.fit(X, y)
    NFOLDS=5
    skf = StratifiedKFold(n_splits=NFOLDS)
    
    # preds=np.zeros((X_test.shape[0],))
    losses=[]
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        # print(X_train.shape,X_val.shape)
        clf=grid_search_cv.best_estimator_
            
        clf.fit(X_train,y_train)
        pval=clf.predict_proba(X_val)[:,1]
        loss=log_loss(y_val,pval)
        losses.append(loss)
    print(np.mean(losses))
    return np.mean(losses)
# evaluate_mdl(X,y)

In [25]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer,PowerTransformer,Normalizer,MaxAbsScaler,KBinsDiscretizer,RobustScaler
continuous=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
target=["target"]
id=["uid"]
categorical=['workclass','marital-status', 'occupation', 'relationship', 'race','sex', 'native-country','education']
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1


data=pd.concat([train,test],ignore_index=True)
data["workclass"]=data["workclass"].apply(str.strip).replace("?",np.nan)
data["workclass"].fillna(data["workclass"].mode()[0],inplace=True)

cnts=data[categorical]["native-country"].value_counts()
smallcnts=list(cnts[cnts<200].index)
data.loc[data["native-country"].isin(smallcnts),"native-country"]="others"




dummies=pd.get_dummies(data[categorical])
dcols=dummies.columns
print(data.shape)
data=data.merge(dummies,how='left',left_index=True,right_index=True)
for col in categorical:
    enc=LabelEncoder()
    data[col]=enc.fit_transform(data[col].values.reshape(-1,1)).flatten()

# data["net-cap"]=data["capital-gain"]-data["capital-loss"]

train=data[data["target"]!=-1]
test=data[data["target"]==-1]
X=train.drop(target+id,axis=1).values
y=train.target.values
"both dummy and label + scaling"
evaluate_mdl(X,y)
# 0.31566125458576005


(48842, 16)
0.31512068804097304


0.31512068804097304

In [23]:
train=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/train.csv")
test=pd.read_csv("../input/ods-mlclass-dubai-2019-03-lecture3-hw/test.csv")
test["target"]=-1
data=pd.concat([train,test],ignore_index=True)
data["workclass"].unique()
data["workclass"].mode()[0]

' Private'